In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20200913.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],Rank your movie choices. You don't have to do a full ranking. [Seventh Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice],What theme should we explore next week? [Eighth Choice]
0,9/8/2020 8:04:20,Killer of Sheep,Edge of the Knife,Losing Ground,Bury The Hatchet,Basquiat,Poetic Justice,NaN,COLOUR IN THE TITLE,PROTESTANTS: FRIEND OR FOE,FEAR OF THE FEMININE,FOOD,TOTALLY SPIES,MARIJUANO IS AN ILLEGAL NARCOTIC,PLANTS: FRIEND OR FOE?,No Theme/Free-for-all
1,9/8/2020 8:30:18,Edge of the Knife,Killer of Sheep,Poetic Justice,Basquiat,Sorry to Bother You,Bury The Hatchet,Losing Ground,FOOD,PLANTS: FRIEND OR FOE?,PROTESTANTS: FRIEND OR FOE,TOTALLY SPIES,COLOUR IN THE TITLE,FEAR OF THE FEMININE,MARIJUANO IS AN ILLEGAL NARCOTIC,NaN
2,9/8/2020 10:03:19,Bury The Hatchet,Poetic Justice,Sorry to Bother You,Losing Ground,Edge of the Knife,Killer of Sheep,Basquiat,PROTESTANTS: FRIEND OR FOE,TOTALLY SPIES,PLANTS: FRIEND OR FOE?,MARIJUANO IS AN ILLEGAL NARCOTIC,FOOD,FEAR OF THE FEMININE,COLOUR IN THE TITLE,No Theme/Free-for-all
3,9/8/2020 10:12:09,Bury The Hatchet,Edge of the Knife,Sorry to Bother You,Poetic Justice,Losing Ground,NaN,NaN,TOTALLY SPIES,COLOUR IN THE TITLE,PROTESTANTS: FRIEND OR FOE,FEAR OF THE FEMININE,FOOD,PLANTS: FRIEND OR FOE?,MARIJUANO IS AN ILLEGAL NARCOTIC,NaN
4,9/8/2020 20:45:46,Poetic Justice,Edge of the Knife,Bury The Hatchet,Basquiat,Killer of Sheep,Sorry to Bother You,Losing Ground,COLOUR IN THE TITLE,FOOD,FEAR OF THE FEMININE,PROTESTANTS: FRIEND OR FOE,TOTALLY SPIES,PLANTS: FRIEND OR FOE?,MARIJUANO IS AN ILLEGAL NARCOTIC,No Theme/Free-for-all
5,9/9/2020 8:43:53,Poetic Justice,Sorry to Bother You,Bury The Hatchet,Killer of Sheep,Edge of the Knife,Basquiat,Losing Ground,PLANTS: FRIEND OR FOE?,FEAR OF THE FEMININE,MARIJUANO IS AN ILLEGAL NARCOTIC,FOOD,TOTALLY SPIES,PROTESTANTS: FRIEND OR FOE,COLOUR IN THE TITLE,No Theme/Free-for-all
6,9/9/2020 10:16:41,Basquiat,Edge of the Knife,Sorry to Bother You,Bury The Hatchet,Killer of Sheep,Losing Ground,Poetic Justice,PROTESTANTS: FRIEND OR FOE,MARIJUANO IS AN ILLEGAL NARCOTIC,FEAR OF THE FEMININE,COLOUR IN THE TITLE,FOOD,PLANTS: FRIEND OR FOE?,TOTALLY SPIES,No Theme/Free-for-all


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Edge of the Knife', 'Killer of Sheep', 'Poetic Justice',
       'Basquiat', 'Sorry to Bother You', 'Bury The Hatchet',
       'Losing Ground'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Edge of the Knife': <Candidate('Edge of the Knife')>,
 'Killer of Sheep': <Candidate('Killer of Sheep')>,
 'Poetic Justice': <Candidate('Poetic Justice')>,
 'Basquiat': <Candidate('Basquiat')>,
 'Sorry to Bother You': <Candidate('Sorry to Bother You')>,
 'Bury The Hatchet': <Candidate('Bury The Hatchet')>,
 'Losing Ground': <Candidate('Losing Ground')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Killer of Sheep, Edge of the Knife, Losing Ground, Bury The Hatchet, Basquiat, Poetic Justice)>,
 <Ballot(Edge of the Knife, Killer of Sheep, Poetic Justice, Basquiat, Sorry to Bother You, Bury The Hatchet, Losing Ground)>,
 <Ballot(Bury The Hatchet, Poetic Justice, Sorry to Bother You, Losing Ground, Edge of the Knife, Killer of Sheep, Basquiat)>,
 <Ballot(Bury The Hatchet, Edge of the Knife, Sorry to Bother You, Poetic Justice, Losing Ground)>,
 <Ballot(Poetic Justice, Edge of the Knife, Bury The Hatchet, Basquiat, Killer of Sheep, Sorry to Bother You, Losing Ground)>,
 <Ballot(Poetic Justice, Sorry to Bother You, Bury The Hatchet, Killer of Sheep, Edge of the Knife, Basquiat, Losing Ground)>,
 <Ballot(Basquiat, Edge of the Knife, Sorry to Bother You, Bury The Hatchet, Killer of Sheep, Losing Ground, Poetic Justice)>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate              Votes  Status
-------------------  -------  --------
Poetic Justice             2  Hopeful
Bury The Hatchet           2  Hopeful
Edge of the Knife          1  Hopeful
Killer of Sheep            1  Hopeful
Basquiat                   1  Rejected
Sorry to Bother You        0  Rejected
Losing Ground              0  Rejected

ROUND 2
Candidate              Votes  Status
-------------------  -------  --------
Edge of the Knife          2  Hopeful
Bury The Hatchet           2  Hopeful
Poetic Justice             2  Hopeful
Killer of Sheep            1  Rejected
Basquiat                   0  Rejected
Sorry to Bother You        0  Rejected
Losing Ground              0  Rejected

ROUND 3
Candidate              Votes  Status
-------------------  -------  --------
Edge of the Knife          3  Hopeful
Bury The Hatchet           2  Hopeful
Poetic Justice             2  Rejected
Killer of Sheep            0  Rejected
Basquiat                   0  Rejected
Sorry to Bothe

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                           Votes  Status
--------------------------------  -------  --------
PROTESTANTS: FRIEND OR FOE              2  Hopeful
COLOUR IN THE TITLE                     2  Hopeful
PLANTS: FRIEND OR FOE?                  1  Hopeful
FOOD                                    1  Hopeful
TOTALLY SPIES                           1  Rejected
FEAR OF THE FEMININE                    0  Rejected
MARIJUANO IS AN ILLEGAL NARCOTIC        0  Rejected
No Theme/Free-for-all                   0  Rejected

ROUND 2
Candidate                           Votes  Status
--------------------------------  -------  --------
COLOUR IN THE TITLE                     3  Hopeful
PROTESTANTS: FRIEND OR FOE              2  Hopeful
FOOD                                    1  Rejected
PLANTS: FRIEND OR FOE?                  1  Rejected
TOTALLY SPIES                           0  Rejected
FEAR OF THE FEMININE                    0  Rejected
MARIJUANO IS AN ILLEGAL NARCOTIC        0  Rejected
No Th